In [4]:
!pip install tmdbsimple -q
!pip3 install rotten_tomatoes_scraper

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1270 sha256=be4b7da1ad110d50dcb6bab965d263d6e1a5cd457ff074bd463af749c847c1b3
  Stored in directory: /root/.cache/pip/wheels/73/2b/cb/099980278a0c9a3e57ff1a89875ec07bfa0b6fcbebb9a8cad3
Successfully built bs4


In [5]:
import tmdbsimple as tmdb
tmdb.API_KEY = 'd24fb5633c876c3450857852f36fd3fb'

In [6]:
from rotten_tomatoes_scraper.rt_scraper import MovieScraper
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [7]:
tmdb.REQUESTS_TIMEOUT = 5

In [8]:
def information_extractor(movie_name, movie_year, hhb_score):

  search = tmdb.Search()
  search_response = search.movie(query=movie_name)

  ## Extracting TMDB ID

  tmdb_movie_id = []

  for s in search.results:
    if int(s['release_date'][0:4]) == movie_year:
      tmdb_movie_id.append(s['id'])
      break
    else:
      continue
  
  ## Extracting TMDB information based on ID

  film_info = tmdb.Movies(tmdb_movie_id[0]).info()
  credits = tmdb.Movies(tmdb_movie_id[0]).credits()


  genres = []

  for genre in film_info['genres']:
    genres.append(genre['name'])

  
  budget = film_info['budget']
  revenue = film_info['revenue']
  runtime = film_info['runtime']
  tmdb_score = film_info['vote_average']
  director = ""
  actors = []

  for person in credits['crew']:
    if person['job'] == "Director":
      director = person['name']
      break
    else:
      continue
  
  first_two_actors = credits['cast'][:2]

  for actor in first_two_actors:
    actors.append(actor['name'])



  # Rotten Tomatoes

  base_url = 'https://www.rottentomatoes.com/m/'
  movie_code_url = "_".join(film_info['title'].strip().lower().split())

  result_url_with_year = base_url + movie_code_url + "_" + str(movie_year)
  result_url_without_year = base_url + movie_code_url

  try:
    try:
      movie_scraper = MovieScraper(movie_url=result_url_with_year)
      movie_scraper.extract_metadata()
      rotten_score = movie_scraper.metadata['Score_Rotten']
    except:
      movie_scraper = MovieScraper(movie_url=result_url_without_year)
      movie_scraper.extract_metadata()
      rotten_score = movie_scraper.metadata['Score_Rotten']
  except:
    rotten_score = "RT Score Unavailable"

  entry = {'Title': film_info['title'],
           'Year': int(movie_year),
           'Budget': budget,
           'Revenue': revenue,
           'Runtime': runtime,
           'Genres': genres,
           'Director': director,
           'Actors': actors,
           'TMDB Score': tmdb_score,
           'RT Score': rotten_score,
           'HHB Score': hhb_score}


  return entry

In [17]:
def query_generator():

  title = str(input('What is the title?\n'))
  year = int(input('What is the release year?\n'))
  hhb_score = str(input('What is the HHB Score?\n'))
  print("***************************************")
  query = {'title': title,
              'year': year,
              'hhb_score': hhb_score}

  return query

In [13]:
def pipeline():

  loop = int(input('How many entries do you want to generate?\n'))

  generated_entries = []

  i = 0

  while i != loop:
    try:
      query = query_generator()
      entry = information_extractor(movie_name = query['title'],
                                    movie_year = query['year'],
                                    hhb_score = query['hhb_score'])
      
      generated_entries.append(entry)
      i = i + 1
    except:
      print('Failed to extract information about this movie.')
      i = i +1
      continue


  generated_dataframe = pd.DataFrame(generated_entries)

  return generated_dataframe

In [110]:
information_extractor("Melancholia", 2011, 'P')

{'Title': 'Melancholia',
 'Year': 2011,
 'Budget': 7400000,
 'Revenue': 15946321,
 'Runtime': 135,
 'Genres': ['Drama', 'Science Fiction'],
 'Director': 'Lars von Trier',
 'Actors': ['Kirsten Dunst', 'Charlotte Gainsbourg'],
 'TMDB Score': 7.2,
 'RT Score': '79',
 'HHB Score': 'P'}

In [18]:
entries = pipeline()

How many entries do you want to generate?
2
What is the title?
assassination of jesse james
What is the release year?
2007
What is the HHB Score?
P
***************************************
What is the title?
melancholia
What is the release year?
2011
What is the HHB Score?
P
***************************************


In [19]:
entries

,Title,Year,Budget,Revenue,Runtime,Genres,Director,Actors,TMDB Score,RT Score,HHB Score
0,The Assassination of Jesse James by the Coward...,2007,30000000,14711793,160,"[Action, Drama, Western]",Andrew Dominik,"[Brad Pitt, Casey Affleck]",7.066,RT Score Unavailable,P
1,Melancholia,2011,7400000,15946321,135,"[Drama, Science Fiction]",Lars von Trier,"[Kirsten Dunst, Charlotte Gainsbourg]",7.200,79,P
